# P2. Les vues SQL

## Définition d'une vue SQL  

Une vue n'est rien de plus qu'une instruction SQLite qui est stockée dans la base de données avec un nom associé. Il s'agit en fait d'une composition d'une table sous la forme d'une requête SQLite prédéfinie.

Une vue peut contenir toutes les lignes d'une table ou des lignes sélectionnées d'une ou plusieurs tables. Une vue peut être créée à partir d'une ou plusieurs tables qui dépendent de la requête SQLite écrite pour créer une vue.

## Code utilitaire

In [1]:
import os
import sqlite3
import pandas as pd   

In [2]:
from pathlib import Path

In [3]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [1]:
# Fonction permettant d'exécuter un requête SQL sur une BDD définie par sa connexion conn
def executer_requete(requete_sql, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(requete_sql)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'execution de la requête")
        print(e)
        return
    cursor.close()

## Example de vue SQL: v_albums

La requête suivante permet d'obtenir une liste (simplifiée) des albums / artistes correspondants:  

In [5]:
# Requete sur les tables albums & artists
requete = """
    SELECT AlbumId,
           Title,
           a.Name AS Artist
    FROM albums
    JOIN artists a on albums.ArtistId = a.ArtistId
    ORDER BY Artist;
"""

albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


Le code ci-dessous permet de créer une vue correspondant à la requête ci-dessous :    
Référence vue sous SQLite : https://www.sqlite.org/lang_createview.html

In [43]:
# creation vue albums avec artistes
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist)
    AS
        SELECT AlbumId,
               Title,
               a.Name AS Artist
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

Une fois la vue créé, on peut la réutiliser dans des requêtes SQL comme une table:  

In [44]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


In [45]:
# Utilisation de la vue v_albums avec une clause WHERE
albums = pd.read_sql_query("SELECT * FROM v_albums WHERE Artist = 'U2';", conn)
albums

,AlbumId,Title,Artist
0,232,Achtung Baby,U2
1,233,All That You Can't Leave Behind,U2
2,234,B-Sides 1980-1990,U2
3,235,How To Dismantle An Atomic Bomb,U2
4,255,Instant Karma: The Amnesty International Campa...,U2
5,236,Pop,U2
6,237,Rattle And Hum,U2
7,238,The Best Of 1980-1990,U2
8,239,War,U2
9,240,Zooropa,U2


In [46]:
# Utilisation de la vue v_albums avec une autre jointure
requete = """
    SELECT a.AlbumId,
           a.Title,
           a.Artist,
           count(t.TrackId) as Tracks
    FROM v_albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    GROUP BY 1, 2, 3
    ORDER BY 3, 2;
"""
albums = pd.read_sql_query(requete, conn)
albums.head()

,AlbumId,Title,Artist,Tracks
0,1,For Those About To Rock We Salute You,AC/DC,10
1,4,Let There Be Rock,AC/DC,8
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1
3,267,Worlds,Aaron Goldberg,1
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2


In [49]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Question: Avantages & Inconvénients des vues SQL

Quelles sont les avantages et inconvénients des vues SQL ?  

Permet d'éviter un trop grand nombre de jointures

---

## Exercices de création / mise en oeuvre de vues

### Vue 1: caractéristiques complètes des Albums  
A partir de l'exemple de vue ci-dessus, créer et exploiter une nouvelle vue v_albums reprenant pour chaque album:  
- AlbumId
- Title
- Artist
- nb de pistes
- Durée totale de l'album (en minutes)
- Prix de l'album

In [40]:
creation_vue_albums2 = """
   CREATE VIEW v2_albums (
        ID,
        Titre,
        Groupe,
        "Nbre pistes",
        Durée,
        Prix)
    AS
        SELECT 
           a.AlbumId AS "ID",
           a.Title AS "Title",
           b.Name AS "Groupe", 
           count(a.Title) AS "Nbre pistes", 
           COALESCE((SUM(c.Milliseconds)/60000),0) AS "Durée", 
           SUM(c.UnitPrice) AS "Prix"
       FROM albums a
       LEFT JOIN artists b ON a.ArtistId=b.ArtistId
       LEFT JOIN tracks c ON a.AlbumId=c.AlbumId
       GROUP BY a.Title
"""

executer_requete(creation_vue_albums2, conn)

In [41]:
albums1 = pd.read_sql_query("SELECT * FROM v2_albums LIMIT 10;", conn)
albums1.head(5)

,ID,Titre,Groupe,Nbre pistes,Durée,Prix
0,156,...And Justice For All,Metallica,9,65,8.91
1,257,20th Century Masters - The Millennium Collecti...,Scorpions,12,57,11.88
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1,3,0.99
3,94,A Matter of Life and Death,Iron Maiden,11,79,10.89
4,95,A Real Dead One,Iron Maiden,12,59,11.88


In [42]:
albums2 = pd.read_sql_query("SELECT * FROM v2_albums WHERE Groupe = 'Foo Fighters';", conn)
albums2

,ID,Titre,Groupe,Nbre pistes,Durée,Prix
0,79,In Your Honor [Disc 1],Foo Fighters,10,40,9.90
1,80,In Your Honor [Disc 2],Foo Fighters,10,43,9.90
2,81,One By One,Foo Fighters,11,55,10.89
3,82,The Colour And The Shape,Foo Fighters,13,46,12.87


In [47]:
requete = """
    SELECT 
       ID,
       Titre,
       t.ArtistId as "ID Groupe",
       Groupe
    FROM v2_albums a
    LEFT JOIN artists t ON a.Groupe=t.Name
    ORDER BY "ID Groupe" ASC
"""
albums3 = pd.read_sql_query(requete, conn)
albums3.head()

,ID,Titre,ID Groupe,Groupe
0,1,For Those About To Rock We Salute You,1,AC/DC
1,4,Let There Be Rock,1,AC/DC
2,2,Balls to the Wall,2,Accept
3,3,Restless and Wild,2,Accept
4,5,Big Ones,3,Aerosmith


In [48]:
executer_requete("DROP VIEW IF EXISTS v2_albums;", conn)

### Vue 2: caractéristiques détaillées des pistes
Créer et exploiter une nouvelle vue v_tracks reprenant pour chaque piste:  
- TrackId
- Name
- Genre
- Album
- Artist
- Composer
- Temps (en seconde)
- MediaType

In [34]:
creation_vue_tracks = """
   CREATE VIEW v_tracks (
        ID,
        Morceau,
        Album,
        Groupe,
        Genre,
        Compositeur,
        Durée,
        Média)
    AS
        SELECT 
           a.TrackId AS "ID",
           a.Name AS "Morceau",
           b.Title AS "Album", 
           c.Name AS "Groupe", 
           b2.Name AS "Genre",
           COALESCE((a.Composer), "Inconnu") AS "Compositeur",
           COALESCE((SUM(a.Milliseconds)/1000),0) AS "Durée",
           b3.Name AS "Média"
       FROM tracks a
       LEFT JOIN albums b ON a.AlbumId=b.AlbumId
       LEFT JOIN artists c ON b.ArtistId=c.ArtistId
       LEFT JOIN genres b2 ON a.GenreId=b2.GenreId 
       LEFT JOIN media_types b3 ON a.MediaTypeId=b3.MediaTypeId
       GROUP BY a.TrackId
"""

executer_requete(creation_vue_tracks, conn)

In [35]:
tracks1 = pd.read_sql_query("SELECT * FROM v_tracks LIMIT 10;", conn)
tracks1.head(5)

,ID,Morceau,Album,Groupe,Genre,Compositeur,Durée,Média
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,Rock,"Angus Young, Malcolm Young, Brian Johnson",343,MPEG audio file
1,2,Balls to the Wall,Balls to the Wall,Accept,Rock,Inconnu,342,Protected AAC audio file
2,3,Fast As a Shark,Restless and Wild,Accept,Rock,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230,Protected AAC audio file
3,4,Restless and Wild,Restless and Wild,Accept,Rock,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252,Protected AAC audio file
4,5,Princess of the Dawn,Restless and Wild,Accept,Rock,Deaffy & R.A. Smith-Diesel,375,Protected AAC audio file


In [38]:
tracks2 = pd.read_sql_query("SELECT * FROM v_tracks WHERE Album = 'Misplaced Childhood';", conn)
tracks2

,ID,Morceau,Album,Groupe,Genre,Compositeur,Durée,Média
0,1745,Pseudo Silk Kimono,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",134,MPEG audio file
1,1746,Kayleigh,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",234,MPEG audio file
2,1747,Lavender,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",153,MPEG audio file
3,1748,Bitter Suite: Brief Encounter / Lost Weekend /...,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",356,MPEG audio file
4,1749,Heart Of Lothian: Wide Boy / Curtain Call,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",366,MPEG audio file
5,1750,Waterhole (Expresso Bongo),Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",133,MPEG audio file
6,1751,Lords Of The Backstage,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",112,MPEG audio file
7,1752,Blind Curve: Vocal Under A Bloodlight / Passin...,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",569,MPEG audio file
8,1753,Childhoods End?,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",272,MPEG audio file
9,1754,White Feather,Misplaced Childhood,Marillion,Rock,"Kelly, Mosley, Rothery, Trewaves",143,MPEG audio file


In [39]:
executer_requete("DROP VIEW IF EXISTS v_tracks;", conn)

### Vue 3: Vue augmentée invoices
A. Créer une nouvelle vue v_invoices à partir de la table invoices ajoutant 3 nouveaux attributs basés sur l'attribut `InvoiceDate`:  
- semaine de la transaction
- mois de la transaction
- année de la transaction

In [ ]:
creation_vue_invoices = """
   CREATE VIEW v_invoices (
        ID,
        Morceau,
        Album,
        Groupe,
        Genre,
        Compositeur,
        Durée,
        Média)
    AS
        SELECT 
           a.TrackId AS "ID",
           a.Name AS "Morceau",
           b.Title AS "Album", 
           c.Name AS "Groupe", 
           b2.Name AS "Genre",
           COALESCE((a.Composer), "Inconnu") AS "Compositeur",
           COALESCE((SUM(a.Milliseconds)/1000),0) AS "Durée",
           b3.Name AS "Média"
       FROM tracks a
       LEFT JOIN albums b ON a.AlbumId=b.AlbumId
       LEFT JOIN artists c ON b.ArtistId=c.ArtistId
       LEFT JOIN genres b2 ON a.GenreId=b2.GenreId 
       LEFT JOIN media_types b3 ON a.MediaTypeId=b3.MediaTypeId
       GROUP BY a.TrackId
"""

executer_requete(creation_vue_invoices, conn)

In [ ]:
select DateTimeCol, DATEPART(YEAR, DateTimeCol) as YearCol, DATEPART(MONTH, DateTimeCol) as MonthCol, DATEPART(DAY, DateTimeCol) as DayCol 
into NewTable
from OldTable

B. Exploiter la vue v_invoices ainsi crée pour déterminer les ventes annuelles par pays.

### Vue 4: A vous de jouer
Créer et exploiter une nouvelle vue de votre choix  